In [1]:
#%pip install --upgrade pip
#%pip install --disable-pip-version-check \
#    torch==1.13.1 \
#    torchdata==0.5.1 --quiet
#
#%pip install \
#    transformers==4.27.2 \
#    datasets==2.11.0 \
#    evaluate==0.4.0 \
#    rouge_score==0.1.2 \
#    loralib==0.1.1 \
#    peft==0.3.0 --quiet
#!pip install evaluate rouge_score loralib peft

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [3]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name='google/flan-t5-base'
#model_name='google/flan-t5-large'
#model_name='google/flan-t5-xl'dddd
#model_name='google/flan-t5-xxl'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


In [6]:
original_model = original_model.to(device)
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt').to(device)
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:

-------------------------------------------------------------------

In [7]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return example


In [8]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [9]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'labels'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'labels'],
        num_rows: 1500
    })
})

In [10]:
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

In [11]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1500
    })
})

In [12]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (12460, 2)
Validation: (500, 2)
Test: (1500, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1500
    })
})


In [13]:
#output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'
output_dir = "./dialogue-summary-training"

training_args = TrainingArguments(
    output_dir=output_dir,
    #learning_rate=1e-5,
    learning_rate=1e-4,
    num_train_epochs=1,
    #num_train_epochs=45,
    #num_train_epochs=15,
    weight_decay=0.01,
    logging_steps=100,
    #max_steps=1
    #max_steps=100
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [14]:
trainer.train()
#trainer.train(resume_from_checkpoint=True)

Step,Training Loss
100,9.797800
200,1.424400
300,0.661800
400,0.421600
500,0.328800
600,0.279500
700,0.264900
800,0.228200
900,0.231700
1000,0.223800


TrainOutput(global_step=1558, training_loss=0.9709720244303595, metrics={'train_runtime': 290.5668, 'train_samples_per_second': 42.882, 'train_steps_per_second': 5.362, 'total_flos': 8532076611502080.0, 'train_loss': 0.9709720244303595, 'epoch': 1.0})

In [15]:
instruct_model_path="./dialogue-summary-checkpoint-local"

trainer.model.save_pretrained(instruct_model_path)
tokenizer.save_pretrained(instruct_model_path)

('./dialogue-summary-checkpoint-local/tokenizer_config.json',
 './dialogue-summary-checkpoint-local/special_tokens_map.json',
 './dialogue-summary-checkpoint-local/tokenizer.json')

In [16]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("./dialogue-summary-checkpoint-local", torch_dtype=torch.bfloat16)
#instruct_model = AutoModelForSeq2SeqLM.from_pretrained("./flan-dialogue-summary-checkpoint", torch_dtype=torch.bfloat16)
#instruct_model = AutoModelForSeq2SeqLM.from_pretrained("./fine-tune-test/dialogue-summary-checkpoint-local", torch_dtype=torch.bfloat16)

In [17]:
instruct_model = instruct_model.to(device)

In [18]:
#index = 200
index = 40

dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to(device)

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
#Person1# is off of the train.
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
#Person1# is late.


In [19]:
rouge = evaluate.load('rouge')

In [20]:
#dialogues = dataset['test'][0:10]['dialogue']
dialogues = dataset['test'][0:20]['dialogue']
#human_baseline_summaries = dataset['test'][0:10]['summary']
human_baseline_summaries = dataset['test'][0:20]['summary']

original_model_summaries = []
instruct_model_summaries = []

for _, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    #input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)
    
zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries))
 
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries'])
df

,human_baseline_summaries,original_model_summaries,instruct_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,#Person1# takes a memo from Ms.W Dawson. #Pers...,#Person1# asks Ms. Dawson to take a dictation ...
1,In order to prevent employees from wasting tim...,Memos to employees.,#Person1# asks Ms. Dawson to take a dictation ...
2,Ms. Dawson takes a dictation for #Person1# abo...,#Person1# is a senior and wants to take a dict...,#Person1# asks Ms. Dawson to take a dictation ...
3,#Person2# arrives late because of traffic jam....,#Person1# is stuck in traffic again. #Person1#...,#Person1# is stuck in traffic. #Person2# think...
4,#Person2# decides to follow #Person1#'s sugges...,#Person2# is finally here. #Person1# is a form...,#Person1# is stuck in traffic. #Person2# think...
5,#Person2# complains to #Person1# about the tra...,#Person1# is finally here.,#Person1# is stuck in traffic. #Person2# think...
6,#Person1# tells Kate that Masha and Hero get d...,#Person1# is a divorcee.,#Person1# tells #Person2# that Masha and Hero ...
7,#Person1# tells Kate that Masha and Hero are g...,#Person1# tells #Person1# that #Person2# is th...,#Person1# tells #Person2# that Masha and Hero ...
8,#Person1# and Kate talk about the divorce betw...,#Person1# and #Person2# are getting divorced.,#Person1# tells #Person2# that Masha and Hero ...
9,#Person1# and Brian are at the birthday party ...,#Person1# is Brian's birthday.,#Person1# asks Brian to have a dance with #Per...


In [21]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': 0.271809963917092, 'rouge2': 0.06325247893213615, 'rougeL': 0.22200634021494214, 'rougeLsum': 0.2222869000073402}
INSTRUCT MODEL:
{'rouge1': 0.32072643093910524, 'rouge2': 0.1187067123648013, 'rougeL': 0.27282005991978125, 'rougeLsum': 0.2713663905240295}


In [22]:
#results = pd.read_csv("data/dialogue-summary-training-results.csv")
#results = pd.read_csv("git-projects/Generative-AI-with-LLMs/Week-2/data/dialogue-summary-training-results.csv")
results = pd.read_csv("/scratch/qualis/git-projects/Generative-AI-with-LLMs/Week-2/data/dialogue-summary-training-results.csv")

human_baseline_summaries = results['human_baseline_summaries'].values
original_model_summaries = results['original_model_summaries'].values
instruct_model_summaries = results['instruct_model_summaries'].values

print(len(instruct_model_summaries))

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)


instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

1500
ORIGINAL MODEL:
{'rouge1': 0.2334158581572823, 'rouge2': 0.07603964187010573, 'rougeL': 0.20145520923859048, 'rougeLsum': 0.20145899339006135}
INSTRUCT MODEL:
{'rouge1': 0.42161291557556113, 'rouge2': 0.18035380596301792, 'rougeL': 0.3384439349963909, 'rougeLsum': 0.33835653595561666}


In [23]:
print("Absolute percentage improvement of INSTRUCT MODEL over HUMAN BASELINE")

improvement = (np.array(list(instruct_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(instruct_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of INSTRUCT MODEL over HUMAN BASELINE
rouge1: 18.82%
rouge2: 10.43%
rougeL: 13.70%
rougeLsum: 13.69%


In [24]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [25]:
peft_model = get_peft_model(original_model, 
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.41%


In [26]:
output_dir = "./peft-dialogue-summary-training"

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=2,
    logging_steps=500,
    #max_steps=1    
)
    
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
)

In [27]:
peft_trainer.train()

Step,Training Loss
500,0.125900
1000,0.106300
1500,0.103300
2000,0.136700
2500,0.099200
3000,0.097900


TrainOutput(global_step=3116, training_loss=0.11095885838715501, metrics={'train_runtime': 533.7273, 'train_samples_per_second': 46.691, 'train_steps_per_second': 5.838, 'total_flos': 1.733507439132672e+16, 'train_loss': 0.11095885838715501, 'epoch': 2.0})

In [28]:
peft_model_path="./peft-dialogue-summary-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

('./peft-dialogue-summary-checkpoint-local/tokenizer_config.json',
 './peft-dialogue-summary-checkpoint-local/special_tokens_map.json',
 './peft-dialogue-summary-checkpoint-local/tokenizer.json')

In [29]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
#peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large", torch_dtype=torch.bfloat16)
#tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

peft_model = PeftModel.from_pretrained(peft_model_base, 
                                       './peft-dialogue-summary-checkpoint-local/',  
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

In [30]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 0
all model parameters: 251116800
percentage of trainable model parameters: 0.00%


In [31]:
peft_model = peft_model.to(device)

index = 200
dialogue = dataset['test'][index]['dialogue']
baseline_human_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to(device)

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
#Person2# wants to upgrade the system and #Person1# suggests adding a painting program to the software. #Person2# suggests adding a CD-ROM drive and #Person2# asks for a CD-ROM drive.
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
#Person1# asks #Person2# what they would need to upgrade their system.
---------------------------------------------------------------------------------------------------
PEFT MODEL: #Person1# suggests upgrading #Person2#'s system and hardware. #Person1# suggests adding a painting program to #Person2#'s software. #Person1# suggests upgrading your hardware. #Person2# migh

In [32]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    input_ids = input_ids.to(device)

    human_baseline_text_output = human_baseline_summaries[idx]
    
    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_summaries.append(original_model_text_output)
    instruct_model_summaries.append(instruct_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries, peft_model_summaries))
 
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries', 'peft_model_summaries'])
df

,human_baseline_summaries,original_model_summaries,instruct_model_summaries,peft_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,Ms. Dawson tells #Person1# that the company wi...,#Person1# asks Ms. Dawson to take a dictation ...,Ms. Dawson takes a dictation for #Person1#. Th...
1,In order to prevent employees from wasting tim...,#Person1# asks Ms. Dawson to take a dictactum ...,#Person1# asks Ms. Dawson to take a dictation ...,Ms. Dawson takes a dictation for #Person1#. Th...
2,Ms. Dawson takes a dictation for #Person1# abo...,#Person1# asks Ms. Dawson to take a dictation ...,#Person1# asks Ms. Dawson to take a dictation ...,Ms. Dawson takes a dictation for #Person1#. Th...
3,#Person2# arrives late because of traffic jam....,#Person2# gets stuck in traffic and #Person1# ...,#Person1# is stuck in traffic. #Person2# think...,#Person2# got stuck in traffic again. #Person1...
4,#Person2# decides to follow #Person1#'s sugges...,#Person2# got stuck in traffic again and think...,#Person1# is stuck in traffic. #Person2# think...,#Person2# got stuck in traffic again. #Person1...
5,#Person2# complains to #Person1# about the tra...,#Person2# got stuck in traffic because of the ...,#Person1# is stuck in traffic. #Person2# think...,#Person2# got stuck in traffic again. #Person1...
6,#Person1# tells Kate that Masha and Hero get d...,#Person1# tells Kate that Masha and Hero are g...,#Person1# tells #Person2# that Masha and Hero ...,Kate and Masha are getting divorced. They are ...
7,#Person1# tells Kate that Masha and Hero are g...,#Person1# tells Kate that Masha and Hero are d...,#Person1# tells #Person2# that Masha and Hero ...,Kate and Masha are getting divorced. They are ...
8,#Person1# and Kate talk about the divorce betw...,#Person1# tells Kate that Masha and Hero are g...,#Person1# tells #Person2# that Masha and Hero ...,Kate and Masha are getting divorced. They are ...
9,#Person1# and Brian are at the birthday party ...,Brian remembers his birthday and invites #Pers...,#Person1# asks Brian to have a dance with #Per...,Brian has a birthday party. He has a dance wit...


In [33]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.4359737656212346, 'rouge2': 0.16319618737426955, 'rougeL': 0.31930290143638673, 'rougeLsum': 0.32062083411483333}
INSTRUCT MODEL:
{'rouge1': 0.348525009991645, 'rouge2': 0.162649757771709, 'rougeL': 0.3069148984797829, 'rougeLsum': 0.30855365052228667}
PEFT MODEL:
{'rouge1': 0.3548804222757782, 'rouge2': 0.09119202529496646, 'rougeL': 0.272425363499192, 'rougeLsum': 0.27078623487877557}


In [34]:
human_baseline_summaries = results['human_baseline_summaries'].values
original_model_summaries = results['original_model_summaries'].values
instruct_model_summaries = results['instruct_model_summaries'].values
peft_model_summaries     = results['peft_model_summaries'].values

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.2334158581572823, 'rouge2': 0.07603964187010573, 'rougeL': 0.20145520923859048, 'rougeLsum': 0.20145899339006135}
INSTRUCT MODEL:
{'rouge1': 0.42161291557556113, 'rouge2': 0.18035380596301792, 'rougeL': 0.3384439349963909, 'rougeLsum': 0.33835653595561666}
PEFT MODEL:
{'rouge1': 0.40810631575616746, 'rouge2': 0.1633255794568712, 'rougeL': 0.32507074586565354, 'rougeLsum': 0.3248950182867091}


In [35]:
print("Absolute percentage improvement of PEFT MODEL over HUMAN BASELINE")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over HUMAN BASELINE
rouge1: 17.47%
rouge2: 8.73%
rougeL: 12.36%
rougeLsum: 12.34%


In [36]:
print("Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(instruct_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL
rouge1: -1.35%
rouge2: -1.70%
rougeL: -1.34%
rougeLsum: -1.35%


In [37]:
#output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'
output_dir = "./dialogue-summary-training-instruct"

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    #num_train_epochs=45,
    #num_train_epochs=15,
    weight_decay=0.01,
    logging_steps=10,
    #max_steps=1
    max_steps=100
)

trainer = Trainer(
    #model=original_model,
    model = instruct_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)
trainer.train()

Step,Training Loss
10,0.211700
20,0.239300
30,0.208300
40,0.214800
50,0.193200
60,0.209200
70,0.217300
80,0.222500
90,0.208900
100,0.227700


TrainOutput(global_step=100, training_loss=0.215283203125, metrics={'train_runtime': 17.5241, 'train_samples_per_second': 45.652, 'train_steps_per_second': 5.706, 'total_flos': 547805881958400.0, 'train_loss': 0.215283203125, 'epoch': 0.06})